In [ ]:
!pip install ultralytics

In [ ]:
!gdown 1GSAF_u7Gxz0Wcxjd-7zL1Gd2WfC0s7Zz
!unzip wind-turbine-damage-challenges.zip
!rm wind-turbine-damage-challenges.zip

In [ ]:
import os
import json
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import shutil
from ultralytics import YOLO
import pandas as pd

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
def load_coco_annotations(annotation_path):
    with open(annotation_path, 'r') as f:
        return json.load(f)

def convert_coco_to_yolo(coco_bbox, img_width, img_height):
    x, y, w, h = coco_bbox
    x_center = x + w/2
    y_center = y + h/2

    # Нормализация координат для YOLO формата
    x_center = x_center / img_width
    y_center = y_center / img_height
    w = w / img_width
    h = h / img_height

    return [x_center, y_center, w, h]

def process_image(image_path, annotation_data, type_folder, output_size=640, overlap=0.5):
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    image_id = next(img_info['id'] for img_info in annotation_data['images']
                   if img_info['file_name'] == os.path.basename(image_path))
    annotations = [ann for ann in annotation_data['annotations'] if ann['image_id'] == image_id]

    output_img_dir = Path(f'processed_dataset/images/{type_folder}')
    output_label_dir = Path(f'processed_dataset/labels/{type_folder}')
    output_img_dir.mkdir(parents=True, exist_ok=True)
    output_label_dir.mkdir(parents=True, exist_ok=True)

    step = int(output_size * (1 - overlap))

    counter = 0

    # Реализация скользящего окна
    for y in range(0, img_height - output_size + 1, step):
        for x in range(0, img_width - output_size + 1, step):
            crop = img[y:y+output_size, x:x+output_size]

            yolo_annotations = []

            for ann in annotations:
                bbox = ann['bbox']
                category_id = ann['category_id']-1

                # Проверяем, находится ли bbox в текущем окне
                if (x <= bbox[0] <= x + output_size and
                    y <= bbox[1] <= y + output_size):

                    # Корректируем координаты bbox относительно окна
                    # обрезаем если выходит за границы
                    new_bbox = [
                        bbox[0] - x,
                        bbox[1] - y,
                        min(bbox[2], output_size - (bbox[0] - x)),
                        min(bbox[3], output_size - (bbox[1] - y))
                    ]

                    yolo_bbox = convert_coco_to_yolo(new_bbox, output_size, output_size)

                    # Проверяем, что все координаты в допустимом диапазоне [0, 1]
                    if all(0 <= coord <= 1 for coord in yolo_bbox):
                        yolo_annotations.append(f"{category_id} {' '.join(map(str, yolo_bbox))}")

            if yolo_annotations:
                output_img_path = output_img_dir / f"{Path(image_path).stem}_{counter}.jpg"
                cv2.imwrite(str(output_img_path), crop)

                output_label_path = output_label_dir / f"{Path(image_path).stem}_{counter}.txt"
                with open(output_label_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations))

                counter += 1

def process_dataset(input_dir, annotation_path, type_folder):
    annotation_data = load_coco_annotations(annotation_path)

    for img_path in tqdm(list(Path(input_dir).glob('*.jpg'))):
        process_image(str(img_path), annotation_data, type_folder)


def train_yolov8():
    # Создаем файл конфигурации датасета
    dataset_yaml = """
    path: /content/processed_dataset/
    train: images/train
    val: images/valid
    names:
      0: corrosion
      1: lightning
      2: lightning receptor
      3: missing teeth
      4: patch
    """

    with open('dataset.yaml', 'w') as f:
        f.write(dataset_yaml)

    model = YOLO('yolov8n.pt')

    model.train(data='dataset.yaml', epochs=200, imgsz=640, batch=16, save_period=5, exist_ok=True,project="detect_turbines", name="yolo8-detect", resume=True)

In [ ]:
# Один раз только запускать
process_dataset("Wind Turbine damage/train", "Wind Turbine damage/train/_annotations.coco.json", "train")
process_dataset("Wind Turbine damage/valid", "Wind Turbine damage/valid/_annotations.coco.json", "valid")

100%|██████████| 271/271 [00:16<00:00, 16.04it/s]


In [ ]:
train_yolov8()

## Submit

In [ ]:
!gdown 1ePuyQSNw6FrvfsOcL0h2Z79d-VD2bgwf
!unzip sample_submission_with_errors.csv.zip
!rm sample_submission_with_errors.csv.zip

In [ ]:
def process_image_for_inference(image_path, model, output_size=640, overlap=0.5):
    img = cv2.imread(image_path)
    img_height, img_width = img.shape[:2]

    temp_dir = Path('temp_inference')
    temp_dir.mkdir(exist_ok=True)

    step = int(output_size * (1 - overlap))

    class_names = [
    "corrosion",
    "lightning",
    "lightning receptor",
    "missing teeth",
    "patch"
    ]

    predictions = []

    for y in range(0, img_height - output_size + 1, step):
        for x in range(0, img_width - output_size + 1, step):
            crop = img[y:y+output_size, x:x+output_size]

            temp_img_path = temp_dir / f"{Path(image_path).stem}_temp.jpg"
            cv2.imwrite(str(temp_img_path), crop)

            results = model(temp_img_path, conf=0.25, verbose=False)

            for result in results:
                boxes = result.boxes

                for box in boxes:
                    class_index = int(box.cls[0].cpu().numpy())
                    class_name = class_names[class_index]
                    conf = float(box.conf[0].cpu().numpy())

                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()

                    x1 = float(x1 + x)
                    y1 = float(y1 + y)
                    x2 = float(x2 + x)
                    y2 = float(y2 + y)

                    bbox = [x1, y1, x2 - x1, y2 - y1]

                    predictions.append([f'"{class_name}"'] + bbox + [conf])

    predictions = non_max_suppression(predictions, iou_threshold=0.3)

    shutil.rmtree(temp_dir)

    return predictions

def non_max_suppression(predictions, iou_threshold):
    """
    Применяет NMS к предсказаниям для удаления дублирующихся боксов
    """
    filtered = []
    # Группируем предсказания по классам для более точной фильтрации
    class_predictions = {}

    for pred in predictions:
        class_name = pred[0]
        if class_name not in class_predictions:
            class_predictions[class_name] = []
        class_predictions[class_name].append(pred)

    for class_name, preds in class_predictions.items():
        preds = sorted(preds, key=lambda x: x[5], reverse=True)

        while preds:
            current = preds.pop(0)
            filtered.append(current)

            adaptive_iou = iou_threshold * (1 + current[5]) / 2
            preds = [pred for pred in preds if not check_overlap(current[1:5], pred[1:5], adaptive_iou)]

    return filtered

def check_overlap(box1, box2, threshold):
    """
    Проверяет перекрытие между двумя боксами
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[0] + box1[2], box2[0] + box2[2])
    y2 = min(box1[1] + box1[3], box2[1] + box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)
    area1 = box1[2] * box1[3]
    area2 = box2[2] * box2[3]

    union = area1 + area2 - intersection
    if union <= 0:
        return False

    iou = intersection / union
    return iou > threshold

def create_submission_file(predictions, output_file='submission.csv'):
    df = pd.DataFrame(predictions)
    df.to_csv(output_file, index=False, sep=',')

def main():
    model = YOLO('/content/best_large.pt')

    df_example = pd.read_csv("/content/sample_submission_with_errors.csv")
    images = df_example["image_id"].to_list()

    results = []
    for image_id in tqdm(images):
        img_path = os.path.join("Wind Turbine damage", "test", image_id)
        predictions = process_image_for_inference(str(img_path), model)
        results.append({"image_id": str(image_id), "objects": str(predictions)})

    create_submission_file(results)

if __name__ == "__main__":
    main()

100%|██████████| 151/151 [24:43<00:00,  9.82s/it]
